In [1]:
import os
import re
import sys
import time
import torch
import GPUtil
import numpy as np
from utils import *
import torch.nn as nn
import pennylane as qml
from typing import List
from Hamiltonian import *
from qudit_mapping import *
from qutrit_synthesis import *
from VAE_model import VAEModel
import matplotlib.pyplot as plt
import torch.nn.functional as F
from itertools import combinations
import torch.distributions as dists
from exact_diagonalization import *
from scipy.linalg import norm, orth
from scipy.sparse.linalg import eigsh
from torch.utils.data import DataLoader
from scipy.sparse import csr_matrix, eye

np.set_printoptions(precision=8, linewidth=200)
torch.set_printoptions(precision=8, linewidth=200)

In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt

energy, fidelity, cos_sim_max, cos_sim_mean, cos_sim_min = [], [], [], [], []
with open('./logs/VGON_nqd7_L3_phase_202501.log') as f:
    lines, count = f.readlines(), 0
    for line in lines:
        energy_pattern = r'Energy: (-*\d\.\d+)'
        fidelity_pattern = r'Fidelity: (\d\.\d+)'
        cos_sim_pattern = r'Cos_Sim: \d+\*(\d\.\d+), (\d\.\d+), (-*\d\.\d+)'
        energy_match = re.search(energy_pattern, line)
        fidelity_match = re.search(fidelity_pattern, line)
        cos_sim_match = re.search(cos_sim_pattern, line)
        if energy_match and fidelity_match and cos_sim_match:
            energy.append(float(energy_match.group(1)))
            fidelity.append(float(fidelity_match.group(1)))
            cos_sim_max.append(float(cos_sim_match.group(1)))
            cos_sim_mean.append(float(cos_sim_match.group(2)))
            cos_sim_min.append(float(cos_sim_match.group(3)))

_, axs = plt.subplots(1, 2, figsize=(12, 5))
x = 1 + np.arange(len(energy))
axs[0].plot(x, energy, linewidth=1)
axs[0].set_title('Energy', fontsize=12)
axs[0].axhline(-8.85166577, color='r', linestyle='--')
axs[1].plot(x, fidelity, linewidth=1)
axs[1].set_title('Fidelity', fontsize=12)
if max(fidelity) > 0.9:
    axs[1].axhline(1, color='r', linestyle='--')
    axs[1].axhline(0.95, color='g', linestyle='--')
    axs[1].set_ylim(-0.05, 1.05)
    axs[1].set_yticks(np.linspace(0, 1, 11))
plt.show()

_, axs = plt.subplots(1, 3, figsize=(18, 5))
x = 1 + np.arange(len(cos_sim_max))
axs[0].plot(x, cos_sim_max, linewidth=0.5)
axs[0].set_title('Cos Sim Max', fontsize=12)
axs[1].plot(x, cos_sim_mean, linewidth=0.5)
axs[1].set_title('Cos Sim Mean', fontsize=12)
axs[2].plot(x, cos_sim_min, linewidth=0.5)
axs[2].set_title('Cos Sim Min', fontsize=12)
plt.show()

In [ ]:
import os
import re
from scipy.io import loadmat

path = './mats'
date = '20250103'
for name in sorted(os.listdir(path), reverse=True):
    match = re.search(r'(VGON+)_nqd\d+(_L\d+)*_\d{8}_\d{6}.mat', name)
    if match and date in name:
        task = match.group(1)
        load = loadmat(f'{path}/{name}')
        phase = load['phase'].item()
        energy = load['energy'].item()
        fidelity = load['fidelity'].sum()
        n_train = load['n_train'].item()
        ground_state_energy = load['ground_state_energy'].item()
        energy_gap = energy - ground_state_energy
        print(f'{name}, {phase}, {ground_state_energy:.8f}, {energy:.8f}, {energy_gap:.4e}, {fidelity.sum():.8f}, {n_train}')

In [ ]:
import torch
import GPUtil
import numpy as np
import pennylane as qml
from utils import fidelity
from scipy.io import loadmat
from VAE_model import VAEModel
from Hamiltonian import BBH_model
from itertools import combinations
import torch.distributions as dists
from torch.utils.data import DataLoader
from qudit_mapping import symmetric_decoding
from qutrit_synthesis import NUM_PR, two_qutrit_unitary_synthesis

np.set_printoptions(precision=8, linewidth=200)
torch.set_printoptions(precision=8, linewidth=200)

n_test = 10
name = 'VGON_nqd7_L3_20250103_121746'

if True:
    load = loadmat(f'./mats/{name}.mat')
    theta = load['theta'].item()
    phase = load['phase'].item()
    kl_div = load['kl_div'].item()
    energy = load['energy'].item()
    n_train = load['n_train'].item()
    n_layers = load['n_layers'].item()
    n_qudits = load['n_qudits'].item()
    batch_size = load['batch_size'].item()
    ground_states = load['ground_states']
    fidelity_mean = load['fidelity'].squeeze()
    ground_state_energy = load['ground_state_energy'].item()
    degeneracy = ground_states.shape[0]

    print(f'{name}, {phase}, Fidelity: {fidelity_mean.max():.8f}, KL_div: {kl_div:.4e}, {degeneracy}')
    print(f'Ground State Energy: {ground_state_energy:.8f}, Energy: {energy:.8f}, Gap: {energy-ground_state_energy:.4e}, {n_train}')

    n_qubits = 2 * n_qudits
    n_samples = batch_size * n_test
    n_params = n_layers * (n_qudits - 1) * NUM_PR

    z_dim = 50
    list_z = np.arange(np.floor(np.log2(n_params)), np.ceil(np.log2(z_dim)) - 1, -1)
    h_dim = np.power(2, list_z).astype(int)

    dev = qml.device('default.qubit', n_qubits)
    gpu_memory = gpus[0].memoryUtil if (gpus := GPUtil.getGPUs()) else 1
    if torch.cuda.is_available() and gpu_memory < 0.8 and n_qubits >= 12:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    def qutrit_symmetric_ansatz(params: torch.Tensor):
        for i in range(n_qudits - 1):
            obj = list(range(n_qubits - 2 * i - 4, n_qubits - 2 * i))
            two_qutrit_unitary_synthesis(params[i], obj)

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_state(n_layers: int, params: torch.Tensor):
        params = params.transpose(0, 1).reshape(n_layers, n_qudits - 1, NUM_PR, batch_size)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.state()

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_expval(n_layers: int, params: torch.Tensor, Ham):
        params = params.transpose(0, 1).reshape(n_layers, n_qudits - 1, NUM_PR, batch_size)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.expval(Ham)

    state_dict = torch.load(f'./mats/{name}.pt', map_location=device, weights_only=True)
    model = VAEModel(n_params, z_dim, h_dim).to(device)
    model.load_state_dict(state_dict)
    model.eval()

    data_dist = dists.Uniform(0, 1).sample([n_samples, n_params])
    test_data = DataLoader(data_dist, batch_size=batch_size, shuffle=True, drop_last=True)

    qubit_Ham = BBH_model(n_qudits, theta)
    overlaps = np.empty((0, ground_states.shape[0]))
    for i, batch in enumerate(test_data):
        with torch.no_grad():
            params, _, _ = model(batch.to(device))
            energy = circuit_expval(n_layers, params, qubit_Ham)
            energy_mean = energy.mean()
            energy_gap = energy_mean - ground_state_energy

            states = circuit_state(n_layers, params).detach().cpu().numpy()
            for state in states:
                decoded_state = symmetric_decoding(state, n_qudits)
                overlap = np.array([fidelity(decoded_state, ground_state) for ground_state in ground_states])
                overlaps = np.vstack((overlaps, overlap))

            fidelities = np.array([fidelity(states[ind[0]], states[ind[1]]) for ind in combinations(range(batch_size), 2)])
            print(f'Energy: {energy_mean:.8f}, {energy_gap:.4e}, Fidelity: {fidelities.max():.8f}, {fidelities.min():.8f}, Overlap: {overlap}, {i+1}/{n_test}')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import ptitprince as pt
import matplotlib.pyplot as plt

def plot_raincloud(overlaps: np.ndarray):
    plt.figure(figsize=(7, 6))
    data = {r'$|\varphi_{\!' + f'{i+1}' + r'}\rangle$': overlaps[:, i] for i in range(num_phi)}
    df = pd.DataFrame(data)
    pt.half_violinplot(data=df, scale='count', width=0.8, inner=None, linewidth=1.2, palette=colors)
    sns.stripplot(data=df, jitter=True, palette=colors, size=1.5)
    plt.ylabel('Overlaps', labelpad=10, fontsize=16)
    # plt.yticks(np.linspace(0, 1, 11))
    plt.xlim(-0.8, 3.4)


num_phi = overlaps.shape[1]
colors = ['#74B816', '#339AF0', '#FD7E14', '#F03E3E']
plot_raincloud(overlaps)

In [ ]:
coeffs = np.array([-0.74, -0.26, -0.24, 0.24, 0.26, 0.49]) * np.pi
coeffs = np.append(coeffs, np.arctan(1 / 3))
for theta in coeffs:
    if theta == np.arctan(1 / 3):
        phase = 'arctan(1/3)'
    else:
        phase = f'{theta/np.pi:.2f}π'
    if np.pi / 4 < theta and theta < np.pi / 2:
        print(f'Critical: {phase}')
    elif -3 * np.pi / 4 < theta and theta < -np.pi / 4:
        print(f'Dimerized: {phase}')
    elif -np.pi / 4 < theta and theta < np.pi / 4:
        print(f'Haldane: {phase}')
    elif (np.pi / 2 < theta and theta < np.pi) or (-np.pi < theta and theta < -3 * np.pi / 4):
        print(f'Ferromagnetic: {phase}')
    else:
        print(f'Wrong: {theta:8f}, {phase}')

In [ ]:
def eigensolver(n_qudits: int, k: int, theta: float):
    n_qubits = 2 * n_qudits
    h3 = qutrit_BBH_model(n_qudits, theta)
    h2 = qubit_BBH_model(n_qudits, theta)
    v3 = np.sort(eigsh(h3, k, which='SA', return_eigenvectors=False))
    v2 = np.sort(eigsh(h2, k, which='SA', return_eigenvectors=False))
    print(f'nqd: {n_qudits:2d} {v3}')
    print(f'nqb: {n_qubits:2d} {v2}')


n_qudits, k = 7, 6
for theta in np.array([-0.74, -0.26, -0.24, 0.24, 0.26, 0.49]) * np.pi:
    print(f'Coefficient phase: {theta/np.pi:.2f}π')
    eigensolver(n_qudits, k, theta)
print(f'Coefficient phase: arctan(1/3)')
eigensolver(n_qudits, k, theta=np.arctan(1 / 3))

In [ ]:
import numpy as np
from exact_diagonalization import *
from scipy.sparse.linalg import eigsh


def eigensolver(n_qudits: int, k: int, theta: float):
    n_qubits = 2 * n_qudits
    h3 = qutrit_BBH_model(n_qudits, theta, is_csr=False)
    h2 = qubit_BBH_model(n_qudits, theta, is_csr=False)
    v3 = np.sort(np.linalg.eigvalsh(h3))
    v2 = np.sort(np.linalg.eigvalsh(h2))
    print(f'nqd: {n_qudits:2d}, {v3[:k]}')
    print(f'nqb: {n_qubits:2d}, {v2[:k]}')


n_qudits, k = 6, 8
for theta in np.array([0.32, -0.71, -0.30, -0.16]) * np.pi:
    print(f'Coefficient phase: {theta/np.pi:.2f}π')
    eigensolver(n_qudits, k, theta)
print(f'Coefficient phase: arctan(1/3)')
eigensolver(n_qudits, k, theta=np.arctan(1 / 3))

In [ ]:
import os
import re
import numpy as np
from scipy.io import loadmat

num, date = 0, '2024112'
threshold = np.array([0, 0, 0])
for name in sorted(os.listdir('./mats'), reverse=True):
    match = re.search(r'(VGON_nqd\d+_\d{8}_\d{6}).mat', name)
    if match and date in name:
        load = loadmat(f'./mats/{name}')
        energy = load['energy'].item()
        n_train = load['n_train'].item()
        fidelity_max = load['fidelity_max'].item()
        if 'count' in load:
            count = re.search(r'(\d+)/(\d+)', load['count'].item())
            energy_th, total = int(count.group(1)), int(count.group(2))
            if energy_th > 1000 and energy < -3.99 and fidelity_max < 0.98:
                num += 1
                overlaps = load['overlaps']
                overlap_sum = overlaps.sum(axis=1)
                overlap_th = len(overlap_sum[overlap_sum > 0.995])

                threshold += np.array([overlap_th, energy_th, total])
                energy_str = f'Energy: {energy_th}/{total} = {energy_th/total*100:.2f}%'
                overlap_str = f'Overlaps: {overlap_th}/{total} = {overlap_th/total*100:.2f}%'
                print(f'{num:2d}, {name}, {n_train}, {energy_str}, {overlap_str}')

overlap_th, energy_th, total = threshold
print(f'Energy Threshold: {energy_th}/{total} = {energy_th/total*100:.1f}% {energy_th/total:.8f}')
print(f'Overlaps Threshold: {overlap_th}/{total} = {overlap_th/total*100:.1f}% {overlap_th/total:.8f}')

In [ ]:
import re
import numpy as np
from scipy.io import loadmat

date = '20241210'
threshold = np.array([0, 0, 0])
with open('./logs/VGON_nqd7_generating_202412.log') as f:
    lines = f.readlines()
    for line in lines:
        if 'Load' in line:
            match = re.search(r'Load:.+(VGON_nqd7_' + date + r'_\d{6})\.mat, (\d+/\d+)', line)
        if 'Energy Upper' in line and match:
            energy_upper = float(re.search(r'Energy Upper: (-\d+\.\d+)', line).group(1))
        if '100/100' in line and match:
            name = match.group(1)
            n_train = match.group(2)
            load = loadmat(f'./mats/{name}.mat')
            overlaps = load['overlaps']
            overlap_sum = overlaps.sum(axis=1)
            overlap_th = len(overlap_sum[overlap_sum > 0.995])
            count = re.search(r'(\d+)/(\d+), 100/100', line)
            energy_th, total = int(count.group(1)), int(count.group(2))
            if energy_th > 1000 and energy_upper == -3.99:
                threshold += np.array([overlap_th, energy_th, total])
                energy_str = f'Energy: {energy_th}/{total} = {energy_th/total*100:.2f}%'
                overlap_str = f'Overlaps: {overlap_th}/{total} = {overlap_th/total*100:.2f}%'
                print(f'{name}, {n_train}, {energy_str}, {overlap_str}')
overlap_th, energy_th, total = threshold
print(f'Energy Threshold: {energy_th}/{total} = {energy_th/total*100:.1f}% {energy_th/total:.8f}')
print(f'Overlaps Threshold: {overlap_th}/{total} = {overlap_th/total*100:.1f}% {overlap_th/total:.8f}')

In [ ]:
import re

with open('./logs/VGON_nqd7_degeneracy_202412.log') as f:
    count = 0
    lines = f.readlines()
    for line in lines:
        energy_pattern = r'Energy: (-*\d\.\d+)'
        fidelity_pattern = r'Fidelity: (\d\.\d+)'
        energy_match = re.search(energy_pattern, line)
        fidelity_match = re.search(fidelity_pattern, line)
        if energy_match and fidelity_match:
            energy = float(energy_match.group(1))
            fidelity = float(fidelity_match.group(1))
            if energy < -3.99 and fidelity < 0.98:
                count += 1
                print(count, line.strip())

In [ ]:
import torch
import pennylane as qml
from typing import List


def spin_operator(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    sx = qml.X(obj[0]) + qml.X(obj[1])
    sy = qml.Y(obj[0]) + qml.Y(obj[1])
    sz = qml.Z(obj[0]) + qml.Z(obj[1])
    return sx, sy, sz


def spin_operator2(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    s1 = spin_operator(obj)
    s2 = [i @ j for i in s1 for j in s1]
    return s2


def Hamiltonian(n_qudits: int, beta: float):
    ham1, ham2 = 0, 0
    for i in range(n_qudits - 1):
        obj1 = [2 * i, 2 * i + 1]
        obj2 = [2 * i + 2, 2 * i + 3]
        ham1 += qml.sum(*[spin_operator(obj1)[i] @ spin_operator(obj2)[i] for i in range(3)])
        ham2 += qml.sum(*[spin_operator2(obj1)[i] @ spin_operator2(obj2)[i] for i in range(9)])
    ham = ham1 / 4 - beta * ham2 / 16
    coeffs, obs = qml.simplify(ham).terms()
    coeffs = torch.tensor(coeffs).real
    return qml.Hamiltonian(coeffs, obs)


Hamiltonian(2, -1 / 3) * 24

In [ ]:
x, y = torch.arange(0, 1, 1e-3), []
for cos_sim_max in x:
    if cos_sim_max > 0.9:
        cos_sim_max_coeff = 8
    elif cos_sim_max > 0.8:
        cos_sim_max_coeff = 4
    elif cos_sim_max > 0.7:
        cos_sim_max_coeff = 2
    else:
        cos_sim_max_coeff = 1
    y.append(cos_sim_max_coeff)
plt.xticks(np.linspace(0, 1, 11))
plt.plot(x, y)
plt.grid()

In [ ]:
coeff = 8
cos_sim_lower = 0.0
coeff_funcs = [
    lambda cos_sim_max: coeff * cos_sim_max,
    lambda cos_sim_max: (coeff * cos_sim_max).ceil(),
    lambda cos_sim_max: 2 * (0.5 * coeff * cos_sim_max).ceil(),
    lambda cos_sim_max: 0.5 * (2 * coeff * cos_sim_max).ceil(),
]
cos_sim_max = torch.arange(cos_sim_lower, 1, 1e-3)
for coeff_func in coeff_funcs:
    plt.plot(cos_sim_max, coeff_func(cos_sim_max))
plt.xticks(np.linspace(cos_sim_lower, 1, 11))
plt.yticks(np.linspace(coeff * cos_sim_lower, coeff, 9))
plt.grid()

In [ ]:
coeff = 40
cos_sim_lower = 0.5
coeff_funcs = [
    lambda cos_sim_max: coeff * (cos_sim_max - cos_sim_lower),
    lambda cos_sim_max: (coeff * (cos_sim_max - cos_sim_lower)).ceil(),
    lambda cos_sim_max: 0.5 * (2 * coeff * (cos_sim_max - cos_sim_lower)).ceil(),
    lambda cos_sim_max: coeff * (cos_sim_max - cos_sim_lower) * cos_sim_max,
]
cos_sim_max = torch.arange(cos_sim_lower, 1, 1e-3)
for coeff_func in coeff_funcs:
    plt.plot(cos_sim_max, coeff_func(cos_sim_max))
plt.xticks(np.linspace(cos_sim_lower, 1, 11))
plt.yticks(np.linspace(0, coeff * (1 - cos_sim_lower), 11))
plt.grid()

In [ ]:
coeff = 8
coeff_funcs = [
    lambda cos_sim: coeff * cos_sim,
    lambda cos_sim: coeff * cos_sim.pow(2),
    lambda cos_sim: coeff * (2 * cos_sim - cos_sim.pow(2)),
    lambda cos_sim: coeff / 10 * (10 * cos_sim).ceil(),
]
cos_sim = torch.arange(0, 1, 1e-3)
for coeff_func in coeff_funcs:
    plt.plot(cos_sim, torch.where(cos_sim > 0, coeff_func(cos_sim), 0))
plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.linspace(0, coeff, 11))
plt.grid()

In [ ]:
import time
import torch
import GPUtil
import numpy as np
import pennylane as qml
from torch import Tensor
from VAE_model import VAEModel
from typing import List, Optional
from itertools import combinations
import torch.distributions as dists
from torch.utils.data import DataLoader
from qutrit_synthesis import NUM_PR, two_qutrit_unitary_synthesis
from torch.optim.optimizer import Optimizer, _use_grad_for_differentiable

np.set_printoptions(precision=8, linewidth=200)
torch.set_printoptions(precision=8, linewidth=200)

opt = 'OGD'
n_test = 1
n_qudits = 2
momentum = 0.9
weight_decay = 0
learning_rate = 5e-2


class OGD(Optimizer):

    def __init__(
        self,
        params,
        lr: float = 1e-3,
        momentum: float = 0,
        weight_decay: float = 0,
        maximize: bool = False,
        differentiable: bool = False,
    ):
        if lr < 0.0:
            raise ValueError(f'Invalid learning rate: {lr}')
        if momentum < 0.0:
            raise ValueError(f'Invalid momentum value: {momentum}')
        if weight_decay < 0.0:
            raise ValueError(f'Invalid weight_decay value: {weight_decay}')
        defaults = dict(
            lr=lr,
            momentum=momentum,
            weight_decay=weight_decay,
            maximize=maximize,
            differentiable=differentiable,
        )
        super().__init__(params, defaults)

    def _init_group(self, group, params, grads, momentum_buffer):
        for p in group['params']:
            if p.grad is not None:
                params.append(p)
                grads.append(p.grad)
                if group['momentum'] != 0:
                    state = self.state[p]
                    momentum_buffer.append(state.get('momentum_buffer'))

    @_use_grad_for_differentiable
    def step(self):
        for group in self.param_groups:
            params: List[Tensor] = []
            grads: List[Tensor] = []
            momentum_buffer: List[Optional[Tensor]] = []
            self._init_group(group, params, grads, momentum_buffer)
            orthogonal_gradient_descent(
                params,
                grads,
                momentum_buffer,
                lr=group['lr'],
                momentum=group['momentum'],
                weight_decay=group['weight_decay'],
                maximize=group['maximize'],
            )
            if group['momentum'] != 0:
                # update momentum_buffers in state
                for p, buff in zip(params, momentum_buffer):
                    state = self.state[p]
                    state['momentum_buffer'] = buff


def orthogonal_gradient_descent(
    params: List[Tensor],
    grads: List[Tensor],
    momentum_buffer: List[Optional[Tensor]],
    lr: float,
    momentum: float,
    weight_decay: float,
    maximize: bool,
):
    for i, param in enumerate(params):
        grad = grads[i] if not maximize else -grads[i]
        if weight_decay != 0:
            grad.add_(param, alpha=weight_decay)
            # grad = grad.add(param, alpha=weight_decay)
        if momentum != 0:
            buffer = momentum_buffer[i]
            if buffer is None:
                buffer = torch.clone(grad).detach()
                momentum_buffer[i] = buffer
            else:
                buffer.mul_(momentum).add_(grad, alpha=1)
            grad = buffer
        param.add_(grad, alpha=-lr)
    print(param)
    print(grad)


if True:
    n_layers = 2
    beta = -1 / 3
    batch_size = 16
    energy_coeff, kl_coeff = 1, 1
    energy_tol, kl_tol = 1e-2, 1e-5

    n_qubits = 2 * n_qudits
    n_samples = batch_size * n_test
    n_params = n_layers * (n_qudits - 1) * NUM_PR
    ground_state_energy = -2 / 3 * (n_qudits - 1)
    print(f'Ground State Energy: {ground_state_energy:.6f}')

    z_dim = 50
    list_z = np.arange(np.floor(np.log2(n_params)), np.ceil(np.log2(z_dim)) - 1, -1)
    h_dim = np.power(2, list_z).astype(int)

    dev = qml.device('default.qubit', n_qubits)
    gpu_memory = gpus[0].memoryUtil if (gpus := GPUtil.getGPUs()) else 1
    if torch.cuda.is_available() and gpu_memory < 0.8 and n_qubits >= 12:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    def spin_operator(obj: List[int]):
        if len(obj) != 2:
            raise ValueError(f'The number of object qubits {len(obj)} should be 2')
        sx = qml.X(obj[0]) + qml.X(obj[1])
        sy = qml.Y(obj[0]) + qml.Y(obj[1])
        sz = qml.Z(obj[0]) + qml.Z(obj[1])
        return sx, sy, sz

    def spin_operator2(obj: List[int]):
        if len(obj) != 2:
            raise ValueError(f'The number of object qubits {len(obj)} should be 2')
        s1 = spin_operator(obj)
        s2 = [i @ j for i in s1 for j in s1]
        return s2

    def Hamiltonian(n_qudits: int, beta: float):
        ham1, ham2 = 0, 0
        for i in range(n_qudits - 1):
            obj1 = [2 * i, 2 * i + 1]
            obj2 = [2 * i + 2, 2 * i + 3]
            ham1 += qml.sum(*[spin_operator(obj1)[i] @ spin_operator(obj2)[i] for i in range(3)])
            ham2 += qml.sum(*[spin_operator2(obj1)[i] @ spin_operator2(obj2)[i] for i in range(9)])
        ham = ham1 / 4 - beta * ham2 / 16
        coeffs, obs = qml.simplify(ham).terms()
        coeffs = torch.tensor(coeffs).real
        return qml.Hamiltonian(coeffs, obs)

    def qutrit_symmetric_ansatz(params: torch.Tensor):
        for i in range(n_qudits - 1):
            obj = list(range(n_qubits - 2 * i - 4, n_qubits - 2 * i))
            two_qutrit_unitary_synthesis(params[i], obj)

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_state(n_layers: int, params: torch.Tensor):
        params = params.transpose(0, 1).reshape(n_layers, n_qudits - 1, NUM_PR, batch_size)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.state()

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_expval(n_layers: int, params: torch.Tensor, Ham):
        params = params.transpose(0, 1).reshape(n_layers, n_qudits - 1, NUM_PR, batch_size)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.expval(Ham)

    Ham = Hamiltonian(n_qudits, beta)
    model = VAEModel(n_params, z_dim, h_dim).to(device)

if opt == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
elif opt == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
elif opt == 'AdamW':
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
else:
    optimizer = OGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

torch.manual_seed(42)
data_dist = dists.Uniform(0, 1).sample([n_samples, n_params])
train_data = DataLoader(data_dist, batch_size=batch_size, shuffle=True, drop_last=True)

start = time.perf_counter()
for i, batch in enumerate(train_data):
    model.train()
    optimizer.zero_grad(set_to_none=True)
    params, mean, log_var = model(batch.to(device))

    energy = circuit_expval(n_layers, params, Ham)
    energy_mean = energy.mean()

    kl_div = -0.5 * (1 + log_var - mean.pow(2) - log_var.exp())
    kl_div = kl_div.mean()

    cos_sims = torch.empty((0), device=device)
    for ind in combinations(range(batch_size), 2):
        cos_sim = torch.cosine_similarity(params[ind[0], :], params[ind[1], :], dim=0)
        cos_sims = torch.cat((cos_sims, cos_sim.unsqueeze(0)), dim=0)
    cos_sim_max = cos_sims.max()
    cos_sim_mean = cos_sims.mean()

    coeff = (energy_mean - ground_state_energy).ceil()
    cos_sim_max_coeff = (1 * cos_sim_max).ceil()
    cos_sim_mean_coeff = coeff / 10 * (10 * cos_sim_mean).ceil() if cos_sim_mean > 0 else 0
    loss = energy_coeff * energy_mean + kl_coeff * kl_div + cos_sim_max_coeff * cos_sim_max
    loss.backward()
    optimizer.step()

    t = time.perf_counter() - start
    cos_sim_str = f'Cos_Sim: {cos_sim_max_coeff:.0f}*{cos_sim_max:.6f}, {cos_sim_mean:.6f}'
    print(f'Loss: {loss:.8f}, Energy: {energy_mean:.8f}, KL: {kl_div:.4e}, {cos_sim_str}, {i+1}/{n_test}, {t:.2f}')

In [ ]:
%matplotlib widget
import torch
import numpy as np
import matplotlib.pyplot as plt



def loss_func(x, y):
    return 0.1 * x**2 + 8 * y**2


energy_th, step = 3, 0.1
x = np.arange(-energy_th, energy_th, step)
y = np.arange(-energy_th, energy_th, step)
x, y = np.meshgrid(x, y)
z = loss_func(x, y)
fig = plt.figure(num=1, figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(x, y, z, linewidth=0.5)


def Optimizor(num_iter: int, learning_rate: float, momentum_coeff: float, label: str = None):
    X, Y, Z = [], [], []
    params = torch.tensor([2., 2.]).requires_grad_(True)
    momentum = torch.zeros(params.shape)

    for _ in range(num_iter):
        params.retain_grad()
        loss = loss_func(*params)
        loss.backward()
        X.append(params[0].detach().numpy())
        Y.append(params[1].detach().numpy())
        Z.append(loss.detach().numpy())
        if momentum_coeff != 0:
            momentum = momentum_coeff * momentum + params.grad
            params.grad = momentum
        params = params - learning_rate * params.grad

    ax.plot(X, Y, Z, linewidth=1.5, label=label)
    ax.scatter(X, Y, Z, linewidth=0.5)


n_test = 50
learning_rate = 0.1
Optimizor(n_test, learning_rate, momentum_coeff=0.0, label='no momentum')
Optimizor(n_test, learning_rate, momentum_coeff=0.7, label='momentum=0.7')

ax.view_init(elev=60, azim=0)
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
import torch
import GPUtil
import numpy as np
import pennylane as qml
from typing import List
from utils import fidelity
from scipy.io import loadmat
from VAE_model import VAEModel
from itertools import combinations
import torch.distributions as dists
from torch.utils.data import DataLoader
from qutrit_synthesis import NUM_PR, two_qutrit_unitary_synthesis

np.set_printoptions(precision=15, linewidth=200)
torch.set_printoptions(precision=15, linewidth=200)

n_test = 1
name = 'VGON_nqd7_20241024_185140'
if True:
    match = loadmat(f'./mats/{name}.mat')
    n_qudits = match['n_qudits'].item()
    n_qubits = match['n_qubits'].item()
    batch_size = match['batch_size'].item()

    n_layers = 2
    beta = -1 / 3
    n_samples = batch_size * n_test
    n_params = n_layers * (n_qudits - 1) * NUM_PR

    z_dim = 50
    list_z = np.arange(np.floor(np.log2(n_params)), np.ceil(np.log2(z_dim)) - 1, -1)
    h_dim = np.power(2, list_z).astype(int)

    dev = qml.device('default.qubit', n_qubits)
    gpu_memory = gpus[0].memoryUtil if (gpus := GPUtil.getGPUs()) else 1
    if torch.cuda.is_available() and gpu_memory < 0.8 and n_qubits >= 12:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    def spin_operator(obj: List[int]):
        if len(obj) != 2:
            raise ValueError(f'The number of object qubits {len(obj)} should be 2')
        sx = qml.X(obj[0]) + qml.X(obj[1])
        sy = qml.Y(obj[0]) + qml.Y(obj[1])
        sz = qml.Z(obj[0]) + qml.Z(obj[1])
        return sx, sy, sz

    def spin_operator2(obj: List[int]):
        if len(obj) != 2:
            raise ValueError(f'The number of object qubits {len(obj)} should be 2')
        s1 = spin_operator(obj)
        s2 = [i @ j for i in s1 for j in s1]
        return s2

    def Hamiltonian(n_qudits: int, beta: float):
        ham1, ham2 = 0, 0
        for i in range(n_qudits - 1):
            obj1 = [2 * i, 2 * i + 1]
            obj2 = [2 * i + 2, 2 * i + 3]
            ham1 += qml.sum(*[spin_operator(obj1)[i] @ spin_operator(obj2)[i] for i in range(3)])
            ham2 += qml.sum(*[spin_operator2(obj1)[i] @ spin_operator2(obj2)[i] for i in range(9)])
        ham = ham1 / 4 - beta * ham2 / 16
        coeffs, obs = qml.simplify(ham).terms()
        coeffs = torch.tensor(coeffs).real
        return qml.Hamiltonian(coeffs, obs)

    def qutrit_symmetric_ansatz(params: torch.Tensor):
        for i in range(n_qudits - 1):
            obj = list(range(n_qubits - 2 * i - 4, n_qubits - 2 * i))
            two_qutrit_unitary_synthesis(params[i], obj)

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_state(n_layers: int, params: torch.Tensor):
        params = params.transpose(0, 1).reshape(n_layers, n_qudits - 1, NUM_PR, batch_size)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.state()

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_expval(n_layers: int, params: torch.Tensor, Ham):
        params = params.transpose(0, 1).reshape(n_layers, n_qudits - 1, NUM_PR, batch_size)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.expval(Ham)

    state_dict = torch.load(f'./mats/{name}.pt', map_location=device, weights_only=True)
    model = VAEModel(n_params, z_dim, h_dim).to(device)
    model.load_state_dict(state_dict)
    model.eval()

    data_dist = dists.Uniform(0, 1).sample([n_samples, n_params])
    test_data = DataLoader(data_dist, batch_size=batch_size, shuffle=True, drop_last=True)

    Ham = Hamiltonian(n_qudits, beta)
for i, batch in enumerate(test_data):
    with torch.no_grad():
        param, _, _ = model(batch.to(device))

    cos_sims = torch.empty((0), device=device)
    for ind in combinations(range(batch_size), 2):
        sim = torch.cosine_similarity(param[ind[0], :], param[ind[1], :], dim=0)
        cos_sims = torch.cat((cos_sims, sim.unsqueeze(0)), dim=0)
    print(f'Cos_Sim: {cos_sims.max():.12f}, {cos_sims.mean():.12f}, {cos_sims.min():.12f}')

    state = circuit_state(n_layers, param)
    fidelities = torch.empty((0), device=device)
    for ind in combinations(range(batch_size), 2):
        fidelity = qml.math.fidelity_statevector(state[ind[0]], state[ind[1]])
        fidelities = torch.cat((fidelities, fidelity.unsqueeze(0)), dim=0)
    print(f'Fidelity: {fidelities.max():.12f}, {fidelities.mean():.12f}, {fidelities.min():.12f}')

    energy = circuit_expval(n_layers, param, Ham)
    print(f'Energy: {energy.max():.12f}, {energy.mean():.12f}, {energy.min():.12f}')

    for i, ind in enumerate(combinations(range(batch_size), 2)):
        print(f'{ind}: Cos_Sim: {cos_sims[i]:.12f}, Fidelity: {fidelities[i]:.12f}')

In [ ]:
import time
import torch
import GPUtil
import numpy as np
import pennylane as qml
from typing import List
from VAE_model import VAEModel
from itertools import combinations
import torch.distributions as dists
from torch.utils.data import DataLoader
from qutrit_synthesis import NUM_PR, two_qutrit_unitary_synthesis

np.set_printoptions(precision=15, linewidth=200)
torch.set_printoptions(precision=15, linewidth=200)

n_test = 100
n_qudits = 4
if True:
    n_layers = 2
    batch_size = 1
    n_qubits = 2 * n_qudits
    n_samples = batch_size * n_test
    n_params = n_layers * (n_qudits - 1) * NUM_PR

    z_dim = 50
    list_z = np.arange(np.floor(np.log2(n_params)), np.ceil(np.log2(z_dim)) - 1, -1)
    h_dim = np.power(2, list_z).astype(int)

    dev = qml.device('default.qubit', n_qubits)
    gpu_memory = gpus[0].memoryUtil if (gpus := GPUtil.getGPUs()) else 1
    if torch.cuda.is_available() and gpu_memory < 0.8 and n_qubits >= 12:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    print(f'PyTorch Device: {device}')

    def qutrit_symmetric_ansatz(params: torch.Tensor):
        for i in range(n_qudits - 1):
            obj = list(range(n_qubits - 2 * i - 4, n_qubits - 2 * i))
            two_qutrit_unitary_synthesis(params[i], obj)

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_state_init(n_layers: int, params: torch.Tensor):
        params = params.reshape(n_layers, n_qudits - 1, NUM_PR)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.state()

    @qml.qnode(dev, interface='torch', diff_method='best')
    def circuit_state(n_layers: int, params: torch.Tensor):
        params = params.transpose(0, 1).reshape(n_layers, n_qudits - 1, NUM_PR, batch_size)
        qml.layer(qutrit_symmetric_ansatz, n_layers, params)
        return qml.state()

    param_init = torch.Tensor(np.random.uniform(0, 1, n_params))
    state_init = circuit_state_init(n_layers, param_init).to(device)

    model = VAEModel(n_params, z_dim, h_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    data_dist = dists.Uniform(0, 1).sample([n_samples, n_params])
    train_data = DataLoader(data_dist, batch_size=batch_size, shuffle=True, drop_last=True)

    start = time.perf_counter()
    params = torch.empty((0), device=device)
    states = torch.empty((0), device=device)
    fidelities = torch.empty((0), device=device)
for i, batch in enumerate(train_data):
    model.train()
    optimizer.zero_grad(set_to_none=True)
    param, _, _ = model(batch.to(device))
    params = torch.cat((params, param), dim=0)

    state = circuit_state(n_layers, param)
    states = torch.cat((states, state), dim=0)

    fidelity = qml.math.fidelity_statevector(state_init, state)
    fidelities = torch.cat((fidelities, fidelity), dim=0)

    loss = fidelity.squeeze(0)
    loss.backward()
    optimizer.step()
    if (i + 1) % 10 == 0:
        t = time.perf_counter() - start
        print(f'Loss: {loss:.15f}, {loss:.8e}, {i+1}/{n_test}, {t:.2f}')
f_sort, f_ind = fidelities.sort()
f_sort, f_ind

In [ ]:
def KL_div(p, q):
    p = torch.softmax(p, dim=0)
    q = torch.softmax(q, dim=0)
    return (p * (p.log2() - q.log2())).sum().unsqueeze(0)


def JS_div(p, q):
    p = torch.softmax(p, dim=0)
    q = torch.softmax(q, dim=0)
    return (p * p.log2() / 2 + q * q.log2() / 2 - (p + q) * ((p + q) / 2).log2() / 2).sum().unsqueeze(0)


def H_distance(p, q):
    p = torch.softmax(p, dim=0)
    q = torch.softmax(q, dim=0)
    return ((p.sqrt() - q.sqrt()).pow(2).sum() / 2).sqrt().unsqueeze(0)


KL_divs = torch.empty((0), device=device)
JS_divs = torch.empty((0), device=device)
cos_sims = torch.empty((0), device=device)
H_dist = torch.empty((0), device=device)
for i in range(n_test):
    cos_sim = torch.cosine_similarity(param_init, params[i], dim=0)
    cos_sims = torch.cat((cos_sims, cos_sim.unsqueeze(0)), dim=0)
    KL_divs = torch.cat((KL_divs, KL_div(param_init, params[i])), dim=0)
    JS_divs = torch.cat((JS_divs, JS_div(param_init, params[i])), dim=0)
    H_dist = torch.cat((H_dist, H_distance(param_init, params[i])), dim=0)

In [ ]:
import matplotlib.pyplot as plt


def plot(x: torch.Tensor, label: str):
    plt.plot(np.arange(n_test), x.detach().numpy(), label=label)


plot(f_sort, 'Fidelity')
plot(cos_sims[f_ind], 'Cos_Sim')
plot(KL_divs[f_ind], 'KL_div')
plot(JS_divs[f_ind], 'JS_div')
plot(H_dist[f_ind], 'H_dist')
plt.xticks(np.linspace(0, n_test, 11))
# plt.yscale('log')
plt.legend()
plt.grid()

In [2]:
beta = -1 / 3
n_qudits = 7
n_qubits = 2 * n_qudits
t1 = time.perf_counter()
ham = qutrit_AKLT_model(n_qudits, beta)
t2 = time.perf_counter()
print('Time:', t2 - t1)
eigvals, eigvecs = eigsh(ham, k=4, which='SA')
# eigvals, eigvecs = eigsh(ham, k=4, sigma=-4)
t3 = time.perf_counter()
print('Time:', t3 - t2)
eigvecs = orth(eigvecs)
t4 = time.perf_counter()
print('Time:', t4 - t3)

for i in range(len(eigvals)):
    print(np.allclose(ham @ eigvecs[:, i], eigvals[i] * eigvecs[:, i], atol=5e-14), norm(eigvecs[:, i], 2), eigvals)
for i in combinations(range(len(eigvals)), 2):
    print(i, fidelity(eigvecs[:, i[0]], eigvecs[:, i[1]]))
t5 = time.perf_counter()
print('Time:', t5 - t4)

ED_states = loadmat('./mats/ED_degeneracy.mat')[f'nqd{n_qudits}'][0, 1]

print(np.count_nonzero(eigvecs), end=' ')
eigvecs[np.abs(eigvecs) < 1e-15] = 0
print(np.count_nonzero(eigvecs))
print(eigvals, csr_matrix(eigvecs))

mat_path = f'./mats/ED_degeneracy.mat'
# updatemat(mat_path, {f'nqd{n_qudits}': (eigvals, eigvecs)})

In [ ]:
n_qudits, beta = 7, -1 / 3
Ham = AKLT_model(n_qudits, beta)
h1 = csr_matrix(Ham.matrix())
h2 = qubit_AKLT_model(n_qudits, beta)
h3 = qutrit_AKLT_model(n_qudits, beta)

v1 = np.sort(eigsh(h1, k=6, which='SA', return_eigenvectors=False))
print(v1)
v2 = np.sort(eigsh(h2, k=6, which='SA', return_eigenvectors=False))
print(v2)
v3 = np.sort(eigsh(h3, k=6, which='SA', return_eigenvectors=False))
print(v3)

In [ ]:
start = time.perf_counter()
print('L', end=' 　')
beta_list = [-0.3, -0.2, -0.1, 0.0, 0.4]
for beta in beta_list:
    if beta < 0:
        print(f'β = {beta:.2f}', end='　')
    else:
        print(f'β = +{beta:.2f}', end='　')
print('Time')
for n_qudits in [4, 5, 6, 7, 8, 9]:
    start_nq = time.perf_counter()
    print(n_qudits, end=' 　')
    s1 = qubit_spin_operator(n_qudits)
    s2 = qubit_spin_operator2(n_qudits)
    for beta in beta_list:
        ham = s1 - beta * s2
        eigvals = eigsh(ham, k=4, which='SA', return_eigenvectors=False)
        eigvals = sorted(eigvals)
        vals = np.array(eigvals[:1])
        for v1 in eigvals[1:]:
            for v2 in vals:
                if np.abs(v1 - v2) < 1e-12:
                    break
            else:
                vals = np.append(vals, v1)
        diff = vals[0] - vals[1]
        print(f'{diff:.6f}', end='　')
    end_nq = time.perf_counter()
    print(f'{(end_nq-start_nq):.2f}')
end = time.perf_counter()
total = end - start
if total >= 60:
    print(f'Total time: {total//60:.0f}m{total%60:.2f}s')
else:
    print(f'Total time: {total:.2f}s')

In [ ]:
Sx = {2: np.array([[0, 1], [1, 0]], dtype=CDTYPE) / 2,  \
      3: np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]], dtype=CDTYPE) / np.sqrt(2)}
Sy = {2: np.array([[0, -1j], [1j, 0]], dtype=CDTYPE) / 2, \
      3: np.array([[0, -1j, 0], [1j, 0, -1j], [0, 1j, 0]], dtype=CDTYPE) / np.sqrt(2)}
Sz = {2: np.array([[1, 0], [0, -1]], dtype=CDTYPE) / 2, \
      3: np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]], dtype=CDTYPE)}

dim = 3
print(Sx[dim])
print(Sy[dim])
print(Sz[dim])

S = Sx[dim] + Sy[dim] + Sz[dim]
print(S)